In [ ]:
import os
import eclipse_cat_builder as ecb

# Load in the specific outputs you're packaging

In [ ]:
stage2_fits = 'YourPathHere_calints.fits'
stage3_specdata = 'YourPathHere_SpecData.h5'
stage4_lcdata = 'YourPathHere_LCData.h5'
stage4cal = 'YourPathHere_CalStellarSpec.h5'
stage5_samples = 'YourPathHere_samples_ch0.h5'
stage5_fit = 'YourPathHere_Table_Save_ch0.txt'

eclipse = 1  # Update based on which eclipse observation this is
STAR = 'YourStarHere'  # E.g., 'GJ 3929'
PLANET = 'YourPlanetHere'  # E.g., 'GJ 3929 b'
SRC_DOI = 'LeaveBlankForNow'

HLSPVER = '0.1'  # Increment to '1.0' once approved and ready for publication

outputDirectory = './saves/'  # Specify where you want the output files+figures to be written (folder must already exist)

# Preparing the High-Level Data Product

In [ ]:
# Build a single-eclipse xarray dataset
ds = ecb.build_single_eclipse(stage2_fits, stage3_specdata, stage4cal, stage5_samples, eclipse, STAR, PLANET, SRC_DOI, HLSPVER)
# Write the dataset as an HDF5 file
h5output_fname = ecb.save_single_eclipse_hdf5(ds, outputDirectory)
# Write the dataset as a FITS file for automated metadata ingest
fitsoutput_fname = ecb.hdf5_single_to_fits(h5output_fname)

# Print the dataset to the console so that we can double-check its contents
ds

# Preparing the Intermediate-Level Data Product

In [ ]:
# Build a single-eclipse xarray dataset
ds2 = ecb.build_lightcurve_dataset(stage2_fits, stage3_specdata, stage4_lcdata, stage5_fit, eclipse, STAR, PLANET, SRC_DOI, HLSPVER)
# Write the dataset as an HDF5 file
h5output_fname = ecb.save_lightcurve_hdf5(ds2, outputDirectory)

# Print the dataset to the console so that we can double-check its contents
ds2

# Making output figures for inclusion in the final report

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.stats import sigma_clip

In [ ]:
t_offset = int(np.floor(np.nanmin(ds2.time.values)))

fig, axs = plt.subplot_mosaic('A;B;C', figsize=(10*0.8,7.5*0.8), sharex=True, gridspec_kw={'hspace':0.075})
# Mask ≥40-sigma outliers for a cleaner figure
axs['A'].errorbar(ds2.time-t_offset, sigma_clip(ds2.rawFlux[0], sigma=40), ds2.rawFluxErr[0], fmt='.', c='k', alpha=0.1)
axs['A'].plot(ds2.time-t_offset, ds2.fullModel[0], '-', c='r', lw=1, label='Full Fitted Model', zorder=np.inf)

axs['B'].errorbar(ds2.time-t_offset, ds2.cleanedFlux[0], ds2.rawFluxErr[0], fmt='.', c='k', alpha=0.1)
axs['B'].plot(ds2.time-t_offset, ds2.astroModel[0], '-', c='r', lw=1, label='Fitted Eclipse Model', zorder=np.inf)

axs['C'].errorbar(ds2.time-t_offset, (ds2.rawFlux[0]-ds2.fullModel[0])*1e6, ds2.rawFluxErr[0]*1e6, fmt='.', c='k', alpha=0.1)
axs['C'].axhline(0, c='r', lw=1, zorder=np.inf)

axs['A'].set_title(f'{PLANET} Fiducial Light Curve Fit')
axs['A'].set_ylabel("Raw Flux")
axs['B'].set_ylabel("Cleaned Flux")
axs['C'].set_ylabel("Residuals (ppm)")
axs['C'].set_xlabel(f"Time (BJD_TDB - {t_offset})")
axs['A'].legend(loc=1)
axs['B'].legend(loc=1)

fig.align_ylabels([axs['A'], axs['B'], axs['C']])

plt.savefig(os.path.join(outputDirectory, 'Figure2.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
t_offset = int(np.floor(np.nanmin(ds2.time.values)))

fig, axs = plt.subplot_mosaic('A;B;C;D;E', figsize=(10*0.8,7.5*0.8), sharex=True, gridspec_kw={'hspace':0.1})

for axname, var in zip(list(axs), [ds2.rawFlux,ds2.centroid_x,ds2.centroid_y,ds2.centroid_sx,ds2.centroid_sy]):
    data = np.ma.masked_where(~np.isfinite(var[0]) + ~np.isfinite(ds2.rawFlux[0]), var[0])
    # Mask ≥40-sigma outliers for a cleaner figure
    data = sigma_clip(data, sigma=40)
    axs[axname].plot(ds2.time-t_offset, data, '.', c='k', ms=1)

axs['A'].set_title(f'{PLANET} Raw Measurements')
axs['A'].set_ylabel("Flux")
axs['B'].set_ylabel("$x$")
axs['C'].set_ylabel("$y$")
axs['D'].set_ylabel("$x$ Width")
axs['E'].set_ylabel("$y$ Width")
axs['E'].set_xlabel(f"Time (BJD_TDB - {t_offset})")

fig.align_ylabels([axs['A'], axs['B'], axs['C'], axs['D'], axs['E']])

plt.savefig(os.path.join(outputDirectory, 'Figure1.png'), dpi=300, bbox_inches='tight')
plt.show()